In [ ]:
%%capture
!pip install yfinance --upgrade

In [ ]:
# import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
%matplotlib inline

In [ ]:
# required dates and tickers
start_date = "2020-01-01"
end_date = "2024-12-31"

tickers = ['AMZN', 'AAPL']

In [ ]:
# calculating night returns, log night returns and merging into a single dataaframe
df = {}

for _ticker in tickers:
  data = yf.download(_ticker, start_date, end_date)
  data = data[['Open', 'Close']]
  data['night_returns'] = (data['Open'] - data['Close'].shift(1)) / data['Close'].shift(1)
  data['log_night_returns'] =  np.log(1 + data['night_returns'])
  data = data.dropna()
  df[_ticker] = data

In [ ]:
# transform prev dataaframe into a ticker daataframe i.e. addition of a column ticker
ret = []
for _ticker, data in df.items():
  data['Ticker'] = _ticker
  ret.append(data[['log_night_returns', 'Ticker']])
returns = pd.concat(ret)

In [ ]:
returns

In [ ]:
returns.columns

In [ ]:
# changint the multiindex dataframe to datafram by dropping ticker level
returns = returns.droplevel('Ticker', axis=1)
returns.columns.name=None

In [ ]:
returns.head()

In [ ]:
# calculating portfolio returns and cumulative returns
returns['port_ret'] = returns.groupby(returns.index)['log_night_returns'].mean()
returns['cum_return'] = (1 + returns['port_ret']).cumprod()

In [ ]:
returns.head()

In [ ]:
# calculate sharpe ratio
sharpe_ratio = returns['port_ret'].mean() / (returns['port_ret'].std() * np.sqrt(252))
print(sharpe_ratio)

In [ ]:
# calculate max drawdown
max_drawdown = ((returns['cum_return'] / returns['cum_return'].cummax()) - 1).min()
print(max_drawdown)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(returns['cum_return'], label='Cumulative Return')
plt.title('PnL Curve')
plt.legend()
plt.show()

In [ ]:
# plotting drawdown
plt.figure(figsize=(10, 6))
drawdown = (returns['cum_return'] / returns['cum_return'].cummax()) - 1
plt.plot(drawdown, label='Drawdown', color='red')
plt.title('Max Drawdown')
plt.legend()
plt.show()

In [ ]:
returns.head()

In [ ]:
# calculaating winners and losers
win = (returns['port_ret'] > 0).sum()
losers = (returns['port_ret']<0).sum()

In [ ]:
print(win+losers == returns.shape[0])
print(win, losers)

In [ ]:
# calculating profit facor by using positive and negative returns
pos_ret = returns[returns['port_ret'] > 0]['port_ret'].sum()
neg_ret =  abs(returns[returns['port_ret'] <= 0]['port_ret'].sum())

profit_factor = pos_ret / neg_ret
print(profit_factor)



```
Improvements for this strategy
1. Addition of risk free ratio for sharp ratio adjustment.
2. Add stoploss mechanism.
3. Add machine learning based strategy to predict overnight returns.
4.
```

